In [1]:
from web3 import Web3
from web3.middleware import geth_poa_middleware
import json

# IUiPoolDataProviderV3 contract address
DATA_PROVIDER_CONTRACT_ADDRESS = "0x3De59b6901e7Ad0A19621D49C5b52cC9a4977e52"

# IPoolAddressesProvider contract address
POOL_ADDRESSES_PROVIDER_CONTRACT_ADDRESS = "0x4dd5ab8Fb385F2e12aDe435ba7AFA812F1d364D0"

# User address for fetching user reserves data
USER_ADDRESS = "0x9A560C3ceBB3a8c0f28Efcc43f4D12e942F4eeD7"

# ABI files for IUiPoolDataProviderV3 and IPoolAddressesProvider contracts
# You can get the ABI files from Etherscan or compile the contracts using solc
with open("IUiPoolDataProviderV3_ABI.json") as f:
    DATA_PROVIDER_ABI = json.load(f)
with open("IPoolAddressesProvider_ABI.json") as f:
    POOL_ADDRESSES_PROVIDER_ABI = json.load(f)

w3 = Web3(Web3.HTTPProvider('https://goerli.infura.io/v3/6f900384657d4152b0627543450ce5e7'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

data_provider_contract = w3.eth.contract(
    address=DATA_PROVIDER_CONTRACT_ADDRESS,
    abi=DATA_PROVIDER_ABI
)

pool_addresses_provider_contract = w3.eth.contract(
    address=POOL_ADDRESSES_PROVIDER_CONTRACT_ADDRESS,
    abi=POOL_ADDRESSES_PROVIDER_ABI
)

# 1. Fetch GHO's reserve data
def get_reserves_data():
    reserves_data, base_currency_info = data_provider_contract.functions.getReservesData(
        pool_addresses_provider_contract.address).call()
    return reserves_data, base_currency_info

# 2. Fetch GHO's user reserve data
def get_user_reserves_data(user_address):
    user_reserves_data, user_borrowing_power = data_provider_contract.functions.getUserReservesData(
        pool_addresses_provider_contract.address, user_address).call()
    return user_reserves_data, user_borrowing_power

def get_reserves_list():
    reserves_list = data_provider_contract.functions.getReservesList(
        pool_addresses_provider_contract.address).call()
    return reserves_list

reserves_data, base_currency_info = get_reserves_data()
print("Reserves Data:", reserves_data)
print("Base Currency Info:", base_currency_info)

user_reserves_data, user_borrowing_power = get_user_reserves_data(USER_ADDRESS)
print("User Reserves Data:", user_reserves_data)
print("User Borrowing Power:", user_borrowing_power)

reserves_list = get_reserves_list()
print("Reserves List:")
for reserve in reserves_list:
    print(reserve)

Reserves Data: [('0xE205181Eb3D7415f15377F79aA7769F846cE56DD', 'AAVE', 'AAVE', 18, 5000, 6500, 11000, 0, True, False, False, True, False, 1000000000000000000000000000, 1000000000000000000000000000, 0, 0, 90000000000000000000000000, 1683984480, '0xAC4D92980562Ac11Af46C6C7CEdD7C819C2028D0', '0x1721dDa383B02ec058Ee7B47596F61246eAD0069', '0xCB62E1d181179d1D86D3877e221D1EdE0bDD8841', '0x13Bdaf61b68b917114D4A356c2098D703D4C4aB7', 2380107695761425690824975, 0, 0, 0, 0, 30000000000, '0xFAa450873a4b162D32f545a85e678C13Ca8d4Ae9', 70000000000000000000000000, 3000000000000000000000000000, 0, 0, 90000000000000000000000000, 0, 450000000000000000000000000, False, False, 0, 0, 0, True, 0, 2, 0, 0, 0, 0, 0, 0, '0x0000000000000000000000000000000000000000', '', False), ('0xD77b79BE3e85351fF0cbe78f1B58cf8d1064047C', 'DAI', 'DAI', 18, 7500, 8000, 10500, 1000, True, True, True, True, False, 1000241252314635863066636284, 1001494533949369579862651760, 901268939241428697697776, 5743095913579504993593753, 62871

In [2]:
LENDING_POOL_CONTRACT_ADDRESS = "0xACf4be18be7D4eA90851f90A2816c2022bc5f9D0"

with open("BorrowLogic.json") as f:
    LENDING_POOL_ABI = json.load(f)

lending_pool_contract = w3.eth.contract(
    address=LENDING_POOL_CONTRACT_ADDRESS,
    abi=LENDING_POOL_ABI
)

event_filter = lending_pool_contract.events.Borrow.create_filter(fromBlock=8990000)
borrow_events = event_filter.get_all_entries()
borrowers = set(event.args._user for event in borrow_events)

print("Borrowers:")
for borrower in borrowers:
    print(borrower)
'''
borrow_events = lending_pool_contract.events.Borrow().getLogs(fromBlock=8000000)
print(borrow_events)

def get_past_borrow_events(from_block, to_block):
    borrow_event_signature = w3.sha3(text='Borrow(address,address,address,uint256,uint256,uint256)').hex()
    borrow_event_filter = w3.eth.filter({
        "fromBlock": from_block,
        "toBlock": to_block,
        "address": LENDING_POOL_CONTRACT_ADDRESS,
        "topics": [borrow_event_signature]
    })
    borrow_events_raw = borrow_event_filter.get_all_entries()
    borrow_events = [lending_pool_contract.events.Borrow().processLog(event) for event in borrow_events_raw]
    return borrow_events

# You may need to adjust the range of blocks to scan, depending on the
# specific GHO market and your Ethereum node's access to historical data.
from_block = 8000000  # Replace with the appropriate starting block number
to_block = w3.eth.get_block('latest')

borrow_events = get_past_borrow_events(from_block, to_block)
borrowers = set(event.args._user for event in borrow_events)

print("Borrowers:")
for borrower in borrowers:
    print(borrower)
'''

Borrowers:


'\nborrow_events = lending_pool_contract.events.Borrow().getLogs(fromBlock=8000000)\nprint(borrow_events)\n\ndef get_past_borrow_events(from_block, to_block):\n    borrow_event_signature = w3.sha3(text=\'Borrow(address,address,address,uint256,uint256,uint256)\').hex()\n    borrow_event_filter = w3.eth.filter({\n        "fromBlock": from_block,\n        "toBlock": to_block,\n        "address": LENDING_POOL_CONTRACT_ADDRESS,\n        "topics": [borrow_event_signature]\n    })\n    borrow_events_raw = borrow_event_filter.get_all_entries()\n    borrow_events = [lending_pool_contract.events.Borrow().processLog(event) for event in borrow_events_raw]\n    return borrow_events\n\n# You may need to adjust the range of blocks to scan, depending on the\n# specific GHO market and your Ethereum node\'s access to historical data.\nfrom_block = 8000000  # Replace with the appropriate starting block number\nto_block = w3.eth.get_block(\'latest\')\n\nborrow_events = get_past_borrow_events(from_block, t

In [3]:
gho_token_address = "0xcbE9771eD31e761b744D3cB9eF78A1f32DD99211"
borrowlogic_contract_address = "0xACf4be18be7D4eA90851f90A2816c2022bc5f9D0"

# Set up contract ABI (only the "Borrow" event is needed)
borrowlogic_contract_abi = [
    {
        "anonymous": False,
        "inputs": [
            {"indexed": True, "internalType": "address", "name": "reserve", "type": "address"},
            {"indexed": False, "internalType": "address", "name": "user", "type": "address"},
            {"indexed": True, "internalType": "address", "name": "onBehalfOf", "type": "address"},
            {"indexed": False, "internalType": "uint256", "name": "amount", "type": "uint256"},
            {
                "indexed": False,
                "internalType": "uint256",
                "name": "interestRateMode",
                "type": "uint256",
            },
            {"indexed": False, "internalType": "uint256", "name": "borrowRate", "type": "uint256"},
            {"indexed": True, "internalType": "uint16", "name": "referralCode", "type": "uint16"},
        ],
        "name": "Borrow",
        "type": "event",
    }
]

# Create a contract object
contract = w3.eth.contract(address=borrowlogic_contract_address, abi=borrowlogic_contract_abi)

# Define the block range (use '0' and 'latest' to get all past events)
from_block = 8000000
to_block = 'latest'

# Get past Borrow events
past_borrow_events = contract.events.Borrow.getLogs(fromBlock=from_block, toBlock=to_block, argument_filters={'reserve': gho_token_address})

# Print past Borrow events
for event in past_borrow_events:
    print(f"Event: {event['event']}, User: {event['args']['user']}, On Behalf Of: {event['args']['onBehalfOf']}, Amount: {event['args']['amount']}, Interest Rate Mode: {event['args']['interestRateMode']}, Borrow Rate: {event['args']['borrowRate']}, Referral Code: {event['args']['referralCode']}")


AttributeError: type object 'Borrow' has no attribute 'getLogs'